# FloPy3

## Simple water-table solution with recharge

This problem is an unconfined system with a uniform recharge rate, a horizontal bottom, and flow between constant-head boundaries in column 1 and 100. MODFLOW models cannot match the analytical solution exactly because they do not allow recharge to constant-head cells. Constant-head cells in column 1 and 100 were made very thin (0.1 m) in the direction of flow to minimize the effect of recharge applied to them. The analytical solution for this problem can be written as:

$h = \sqrt{b_{1}^{2} - \frac{x}{L} (b_{1}^{2} - b_{2}^{2}) + (\frac{R x}{K}(L-x))} + z_{bottom}$

where $R$ is the recharge rate, $K$ is the the hydraulic conductivity in the horizontal direction, $b_1$ is the specified saturated thickness at the left boundary, $b_2$ is the specified saturated thickness at the right boundary, $x$ is the distance from the left boundary $L$ is the length of the model domain, and $z_{bottom}$ is the elebation of the bottom of the aquifer.

The model consistes of a grid of 100 columns, 1 row, and 1 layer; a bottom altitude of 0 m; constant heads of 20 and 11m in column 1 and 100, respectively; a recharge rate of 0.001 m/d; and a horizontal hydraulic conductivity of 50 m/d.  The discretization is 0.1 m in the row direction for the constant-head cells (column 1 and 100) and 50 m for all other cells.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import flopy.utils.binaryfile as bf

In [ ]:
#paths to existing output
mf6pth = os.path.join('..', 'test012_WaterTable')

### Function to calculate the analytical solution at specified points in a aquifer

In [ ]:
def analyticalWaterTableSolution(h1, h2, z, R, K, L, x):
    h = np.zeros((x.shape[0]), np.float)
    #dx = x[1] - x[0]
    #x -= dx
    b1 = h1 - z
    b2 = h2 - z
    h = np.sqrt(b1**2 - (x/L)*(b1**2 - b2**2) + (R * x / K) * (L - x)) + z
    return h

## Model parameters for all models. Also calculate the analytical solution.

In [ ]:
#--model dimensions
nlay, nrow, ncol = 1, 1, 100

#--cell spacing
delr = 50.
delc = 1.

#--domain length
L = 5000.

#--boundary heads
h1 = 20.
h2 = 11.

#--ibound
ibound = np.ones((nlay, nrow, ncol), dtype=np.int)

#--starting heads
strt = np.zeros((nlay, nrow, ncol), dtype=np.float)
strt[0, 0, 0] = h1
strt[0, 0, -1] = h2

#--top of the aquifer
top = 25.

#--bottom of the aquifer
botm = 0.

#--hydraulic conductivity
hk = 50.

#--location of cell centroids
x = np.arange(0.0, L, delr) + (delr / 2.)

#--location of cell edges
xa = np.arange(0, L+delr, delr)

#--recharge rate
rchrate = 0.001

#--calculate the head at the cell centroids using the analytical solution function
hac = analyticalWaterTableSolution(h1, h2, botm, rchrate, hk, L, x)

#--calculate the head at the cell edges using the analytical solution functionn
ha = analyticalWaterTableSolution(h1, h2, botm, rchrate, hk, L, xa)

### Read the simulated MODFLOW 6 model results

In [ ]:
#--MODFLOW 6
headfile = os.path.join(mf6pth, 'watertable.hds')
mf6obj = bf.HeadFile(headfile)
times = mf6obj.get_times()
print(times)
head = mf6obj.get_data(totim=times[-1]) 
mf6head = head[0, 0, :]

### Read the simulated MODFLOW-NWT model results

In [ ]:
#--MODFLOW-NWT
headfile = os.path.join(mf6pth, 'mfnwt', 'watertable.hds')
mfobj = bf.HeadFile(headfile)
times = mfobj.get_times()
print(times)
head = mfobj.get_data(totim=times[-1]) 
mfhead = head[0, 0, :]

In [ ]:
d = mf6head - mfhead
print(d.min(), d.max(), d.mean(), d.std())

### Plot the MODFLOW 6 results and compare to the analytical solution

In [ ]:
fig = plt.figure(figsize=(16,6))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
                    wspace=0.25, hspace=0.25)

ax = fig.add_subplot(1, 3, 1)
ax.plot(xa, ha, linewidth=8, color='0.5', label='analytical solution')
ax.plot(x, mf6head, color='red', label='MODFLOW 6')
leg = ax.legend(loc='lower left')
leg.draw_frame(False)
ax.set_xlabel('Horizontal distance, in m')
ax.set_ylabel('Head, in m')

ax = fig.add_subplot(1, 3, 2)
ax.plot(x, mf6head - hac, linewidth=1, color='blue')
ax.set_xlabel('Horizontal distance, in m')
ax.set_ylabel('Error, in m')

ax = fig.add_subplot(1, 3, 3)
ax.plot(x, 100.*(mf6head - hac)/hac, linewidth=1, color='blue')
ax.set_xlabel('Horizontal distance, in m')
ax.set_ylabel('Percent Error')

### Plot the MODFLOW 6 results and compare to MODFLOW-NWT

In [ ]:
fig = plt.figure(figsize=(16,6))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
                    wspace=0.25, hspace=0.25)

ax = fig.add_subplot(1, 3, 1)
ax.plot(x, mfhead, linewidth=8, color='0.5', label='MODFLOW-NWT')
ax.plot(x, mf6head, color='red', label='MODFLOW 6')
leg = ax.legend(loc='lower left')
leg.draw_frame(False)
ax.set_xlabel('Horizontal distance, in m')
ax.set_ylabel('Head, in m')

ax = fig.add_subplot(1, 3, 2)
ax.plot(x, mf6head - mfhead, linewidth=1, color='blue')
ax.set_xlabel('Horizontal distance, in m')
ax.set_ylabel('Error, in m')

ax = fig.add_subplot(1, 3, 3)
ax.plot(x, 100.*(mf6head - mfhead)/mfhead, linewidth=1, color='blue')
ax.set_xlabel('Horizontal distance, in m')
ax.set_ylabel('Percent Error')